# Parlamentarni izbori 2016, or:

## How I Learned to Stop Worrying and Love D'Hondt

&nbsp;

**Autor**: Miran T.

&nbsp;

> Republika Hrvatske podijeljena je u 10 izbornih jedinica. U svakoj izbornoj jedinici se bira po 14 zastupnika.  
Birači koji nemaju prebivalište u Republici Hrvatskoj, glasuju u 11. izbornoj jedinici za dijasporu, i biraju 3 zastupnika.  
Pripadnici nacionalnih manjina odlučuju hoće li glasati u jednoj od deset izbornih jedinica, ili u 12. izbornoj jedinici za dijasporu, u kojoj se bira 8 zastupnika.  
Broj zastupnika koji će biti izabrani u pojedinoj izbornoj jedinici određuje se po [D'Hondtovoj metodi](https://en.wikipedia.org/wiki/D%27Hondt_method), a u raspodijeli zastupničkih mjesta sudjeluju sve izborne liste koje osvoje barem 5% važećih glasova.

Izbori za zastupnike u Hrvatski sabor 2016. održani su 11. rujna 2016. godine. 

U ovom radu prikazat će se rezultati navedenih izbora (za prvih 10 izbornih jedinica - bez manjina i dijaspore) u raznim imaginarnim scenarijima:

- [cijela RH je jedna izborna jedinica](#Cijela-Hrvatska-je-jedna-izborna-jedinica)
- [izborni prag je 3%](#Izborni-prag-je-3%)
- [cijela RH je jedna izborna jedinica i izborni prag je 3%](#Cijela-Hrvatska-je-jedna-izborna-jedinica-i-izborni-prag-je-3%)
- [u svakoj izbornoj jedinici bira se 12 zastupnika](#U-svakoj-izbornoj-jedinici-bira-se-12-zastupnika)
- [korištenje Webster metode](#Kori%C5%A1tenje-Webster-metode)
- [Webster metoda i izborni prag od 3%](#Webster-metoda-i-izborni-prag-od-3%)

&nbsp; 

*Napomena: naslov je referenca na film "Dr. Strangelove" i njime se ne izražava autorova ljubav prema nijednom izbornom sustavu ili metodi.*

&nbsp;

&nbsp;


## Python kod

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (11.0, 7.0)
plt.rcParams['font.size'] = 10
plt.rcParams['figure.titlesize'] = 12
plt.rcParams['figure.titleweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

In [3]:
df = pd.read_csv('./#datasets/hr_izbori.csv', index_col='Izborna jedinica')
df

OSError: File b'./#datasets/hr_izbori.csv' does not exist

In [ ]:
def izracunaj_zastupnike(data=df, izborni_prag=0.05, coeff=1,
                         zastupnika_po_jedinici=14):
    
    def zastupnici_po_jedinici(glasovi):
        presli_prag = glasovi[glasovi > izborni_prag*glasovi.Ukupno].drop('Ukupno')
        quotient = [presli_prag//(coeff*i+1) for i in range(zastupnika_po_jedinici)]
        min_glasova = sorted(np.array(quotient).ravel())[-zastupnika_po_jedinici]
        zastupnici = pd.DataFrame(quotient)
        return zastupnici[zastupnici >= min_glasova].count()
    
    zastupnici_po_jedinicama = []
    if isinstance(data, pd.DataFrame):
        
        for jedinica in data.index.values:
            glasovi = data.ix[jedinica]
            zastupnici = zastupnici_po_jedinici(glasovi)
            zastupnici_po_jedinicama.append(zastupnici)
    else:
        zastupnici_po_jedinicama.append(zastupnici_po_jedinici(data))

    return (pd.DataFrame(zastupnici_po_jedinicama, index=df.index)
            .reindex(columns=df.columns)
            .drop('Ukupno', axis=1)
           )

In [ ]:
def plot_zastupnika(rezultati, usporedi_s=None, tekst='', filename=None):
    cols = 2 if usporedi_s is not None else 1
    fig, axes = plt.subplots(rezultati.shape[0], cols, 
                             figsize=(5*cols, 12), sharex='col')
    axes = axes.ravel()
    
    def plotaj_jedinicu(jedinica, desni_graf=False):
        jedinica.plot(kind='barh', width=1, ax=ax)
        ax.tick_params(axis='both', length=0, which='both')
        
        if not desni_graf:
            ax.set_ylabel('{}. IJ'.format(i))
            ax.set_xlim(0, 8)
            ax.set_xlabel('Broj zastupnika')
        else:
            ax.axvline(x=0, color='k')
            ax.set_yticklabels([])
            ax.set_xlim(-4, 4)
            ax.set_xlabel('Razlika u broju zastupnika')
            
    
    for i, ax in enumerate(axes[::cols], 1):
        izb_jed = rezultati.ix[i].dropna().sort_index(ascending=False)
        plotaj_jedinicu(izb_jed)
    axes[0].set_title('Broj zastupnika po izbornim jedinicama')
        
    if usporedi_s is not None:
        razlike = rezultati.sub(usporedi_s, fill_value=0)
        
        for i, ax in enumerate(axes[1::cols], 1):
            izb_jed = razlike.ix[i].dropna().sort_index(ascending=False)
            plotaj_jedinicu(izb_jed, desni_graf=True)
        
        axes[0].set_title('Broj zastupnika po izbornim jedinicama\n'
                          '({})'.format(tekst))
        axes[1].set_title('Razlika u odnosu na sadašnji sustav')
        fig.subplots_adjust(wspace=0.1)
        
    if filename:
        plt.tight_layout()
        plt.savefig('./figures/hr_izbori_{}_01.png'.format(filename), dpi=70)

In [ ]:
zbroji_zastupnike = lambda x: x.sum().sort_values(na_position='first')


def plot_ukupno_zastupnika(rezultati, usporedi_s=None, tekst='', 
                           xlim=(-4, 4), filename=None):
    def plotaj(graf, ax):
        graf.plot(kind='barh', width=0.5, ax=ax)
        ax.tick_params(axis='both', length=0, which='both')
    
    cols = 2 if usporedi_s is not None else 1
    fig, axes = plt.subplots(1, cols, figsize=(5*cols, 5))

    if cols == 1:
        ax1 = axes
        plotaj(rezultati, ax1)
    else:
        ax1, ax2 = axes
        plotaj(rezultati, ax1)
        razlike = rezultati.sub(usporedi_s.reindex_like(rezultati), fill_value=0)
        plotaj(razlike, ax2)

    for bar, value in zip(ax1.patches, rezultati.values):
        if value > 1:
            x = bar.get_width()
            y = bar.get_y()
            dx = ax1.get_xbound()[1] / 100
            dy = bar.get_height() / 2
            ax1.text(x-dx, y+dy, int(value), va='center', ha='right', color='white')
            
            
    _ = (ax1.set_title('Ukupni broj zastupnika'),
         ax1.set_xlim(0, 60),
         ax1.set_xlabel('Broj zastupnika'),
         ax1.set_ylabel('Stranka')
        )
    
    if cols == 2:
         _ = (ax1.set_title('Ukupni broj zastupnika\n'
                            '({})'.format(tekst)),
              ax2.set_title('Razlika u odnosu na sadašnji sustav'),
              ax2.axvline(x=0, color='k'),
              ax2.set_yticklabels([]),
              ax2.set_xlim(xlim),
              ax2.set_xlabel('Razlika u broju zastupnika'),
              fig.subplots_adjust(wspace=0.1),
             )
    if filename:
        plt.tight_layout()
        plt.savefig('./figures/hr_izbori_{}_02.png'.format(filename), dpi=70)

## Trenutni izborni sustav

Rezultati po izbornim jedinicama i ukupni rezultati dobiveni po trenutno važećim pravilima (D'Hondtova metoda, 5% izborni prag).

In [ ]:
zastupnici = izracunaj_zastupnike()
zastupnici_ukupno = zbroji_zastupnike(zastupnici)
filename = '01_trenutno'

plot_zastupnika(zastupnici, filename=filename)
plot_ukupno_zastupnika(zastupnici_ukupno, filename=filename)

## Cijela Hrvatska je jedna izborna jedinica

U ovom scenariju promatramo kako bi izgledala raspodjela mandata u Saboru da je cijela Hrvatska jedna izborna jedinica u kojoj se bira 140 zastupnika.

In [ ]:
jedna_izborna = df.sum()
jedna_zastupnici = (izracunaj_zastupnike(jedna_izborna, zastupnika_po_jedinici=140)
                    .ix[1]
                    .fillna(0)
                    .sort_values()
                   )
tekst = 'jedna izborna jedinica'
filename = '02_jedna'

plot_ukupno_zastupnika(jedna_zastupnici, zastupnici_ukupno, 
                       tekst=tekst, filename=filename)

Pobjednici ovakvih izbora (u odnosu na trenutne izbore) su MOST i Živi zid, koji osvajaju 25% više zastupničkih mjesta.  
Gubitnici su, logično, stranke koje su uspjele osvojiti mandate samo u pojedinim izbornim jedinicama: IDS, HDDSB i Bandić.

## Izborni prag je 3%

Često se može čuti kako je hrvatski izborni prag od 5% previsok i da bi ga trebalo sniziti kako bi se dala šansa manjim strankama.  
Koliko bi se promijenila slika u Saboru da je izborni prag 3% umjesto dosadašnjih 5%?

In [ ]:
zastupnici_3posto = izracunaj_zastupnike(izborni_prag=0.03)
zastupnici_3posto_ukupno = zbroji_zastupnike(zastupnici_3posto)
tekst = 'izborni prag 3%'
filename = '03_prag3posto'

plot_zastupnika(zastupnici_3posto, zastupnici, tekst=tekst, filename=filename)
plot_ukupno_zastupnika(zastupnici_3posto_ukupno, zastupnici_ukupno, tekst=tekst, filename=filename)

Suprotno nagađanjima, vidimo da smanjenjem izbornog praga na 3% ne bi došlo do nikakvih promjena u Saboru: osvojeni mandati u svim izbornim jedinicama ostaju jednaki kao i kod trenutnog sustava.  
Ono što se može primijetiti (na gornjem grafu) je veliki broj slučajeva gdje stranke prelaze izborni prag, ali nemaju dovoljan broj glasova za osvajanje mandata: Bandić u čak četiri izborne jedinice, Pametno i Živi zid u dvije izborne jedinice, a HDSSB u jednoj.

## Cijela Hrvatska je jedna izborna jedinica i izborni prag je 3%

Kombinacijom prethodna dva scenarija, dobivamo pod
jelu zastupničkih mjesta kada bi cijela Hrvatska bila jedna izborna jedinica uz izborni prag od 3%.

In [ ]:
jedna_izborna = df.sum()
jedna_zastupnici = (izracunaj_zastupnike(jedna_izborna, 
                                         zastupnika_po_jedinici=140, 
                                         izborni_prag=0.03)
                    .ix[1]
                    .fillna(0)
                    .sort_values()
                   )
tekst = 'jedna izborna jedinica i prag 3%'
filename = '04_jedna3posto'

plot_ukupno_zastupnika(jedna_zastupnici, zastupnici_ukupno, 
                       tekst=tekst, filename=filename)

Za razliku od jedne izborne jedinice uz izborni prag od 5%, kada Bandić ostaje bez ijednog mandata, u ovakvom scenariju postaje pobjednik koji je utrostručio broj zastupničkih mjesta.  
Lokalne stranke (IDS, HDSSB) u ovakvom scenariju nemaju šanse.

## U svakoj izbornoj jedinici bira se 12 zastupnika

Kako bi se smanjio ukupni broj zastupnika u Saboru, umjesto dosadašnjih 14 zastupnika po izbornoj jedinici, sada ih se bira samo 12 po jedinici.

In [ ]:
zastupnici_10 = izracunaj_zastupnike(zastupnika_po_jedinici=12)
zastupnici_10_ukupno = zbroji_zastupnike(zastupnici_10)

tekst = '10 zastupnika po jedinici'
filename = '05_10zastupnika'

plot_zastupnika(zastupnici_10, zastupnici, tekst=tekst, filename=filename)
plot_ukupno_zastupnika(zastupnici_10_ukupno, zastupnici_ukupno, 
                       tekst=tekst, xlim=(-9, 0), filename=filename)

Smanjenje broja zastupnika, čak i ako se radi o samo dva zastpunika manje po izbornoj jedinici, najgore pogađa stranke koje su osvojile najviše mandata.  
Tri stranke s najmanjim brojem mandata zadržale bi sve svoje mandate i tako, relativno gledajući, ojačale svoju poziciju u Saboru.

## Korištenje Webster metode

Više o metodi [ovdje](https://en.wikipedia.org/wiki/Webster/Sainte-Lagu%C3%AB_method).

Ukratko:  
Kod D'Hondt metode kvocijenti kojima se dijele glasovi iznose 1, 2, 3, 4, 5, itd.  
Kod Webster metode kvocijenti kojima se dijele glasovi iznose 1, 3, 5, 7, 9, itd. 

Neke od zemalja u kojima se koristi Webster metoda: Norveška, Švedska, Njemačka za izbor u Bundestag, te susjedna Bosna i Hercegovina.

U teoriji, zbog većih kvocijenata, ova metoda bi trebala više pogodovati strankama koje su osvojile manji broj glasova.

In [ ]:
zastupnici_webster = izracunaj_zastupnike(coeff=2)
zastupnici_webster_ukupno = zbroji_zastupnike(zastupnici_webster)
tekst = 'Webster metoda'
filename = '06_webster'

plot_zastupnika(zastupnici_webster, zastupnici, tekst=tekst, filename=filename)
plot_ukupno_zastupnika(zastupnici_webster_ukupno, zastupnici_ukupno, 
                       tekst=tekst, filename=filename)

Dvije najveće stranke osvajaju ukupno tri mandata manje, što i nije velika razlika u odnosu na sadašnji sustav.

## Webster metoda i izborni prag od 3%

Iako bi Webster metoda trebala pogodovati manjim strankama, izborni prag od 5% predstavlja previsoku barijeru za neke od njih.  
Vidjeli smo da [smanjenje izbornog praga na 3% kod D'Hondt metode](#Izborni-prag-je-3%) ne donosi nikakvu promjenu. Što donosi smanjenje izbornog praga na 3% kod Webster metode?

In [ ]:
zastupnici_webster3 = izracunaj_zastupnike(coeff=2, izborni_prag=0.03)
zastupnici_webster3_ukupno = zbroji_zastupnike(zastupnici_webster3)
tekst = 'Webster metoda i prag 3%'
filename = '07_webster3posto'

plot_zastupnika(zastupnici_webster3, zastupnici, tekst=tekst, filename=filename)
plot_ukupno_zastupnika(zastupnici_webster3_ukupno, zastupnici_ukupno, 
                       filename=filename, tekst=tekst, xlim=(-6, 6))

Webster metoda uz izborni prag od 3% ublažava razlike u broju mandata između stranaka, pogodujući strankama s manje glasova.  
Jedino u ovom scenariju stranka Pametno uspijeva postati parlamentarna stranka uz dva zastupnika (iz 1. i 10. izborne jedinice).

&nbsp;

&nbsp;

&nbsp;


In [1]:
from IPython.core.display import HTML
def css_styling(): 
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()